In [3]:
from itertools import product
from math import prod
import random

In [4]:
def is_number(x):
    try: 
        _ = int(x)
        return True
    except:
        return False

def merge_names(name, op):
    args = list(map(str.strip, name.split(op)))
    symbs = [a for a in args if not is_number(a)]
    other = [int(a) for a in args if is_number(a)]
    if op == '+': op = sum
    elif op == '*': op = prod
    symbs.append(str(op(other)))
    return ' + '.join(symbs)

In [5]:
merge_names('d0 + d1 + 3 + 2 +  -1', '+')

'd0 + d1 + 4'

In [6]:
class Symbol:
    def __init__(self, name, rnge=range(1, 10)):
        self.range = set(rnge)
        if self.is_const:
            self.name = str(self.const_value)
        else:
            self.name = name
    
    def __repr__(self):
        return f'{self.name}'
    
    @property
    def is_const(self):
        return len(self.range) == 1
    
    @property
    def const_value(self):
        if not self.is_const: raise ValueError("Is not constant")
        return next(iter(self.range))
    
    @property
    def is_zero(self):
        return self.is_const and self.const_value == 0
    
    @property
    def is_one(self): 
        return self.is_const and self.const_value == 1
    
    def __radd__(self, other):
        if isinstance(other, int):
            other = Symbol(f'{other}', {other})
        if self.is_zero and other.is_zero: return zero
        elif other.is_zero: return self
        elif self.is_zero: return other
        
        res = Symbol(f'({self.name} + {other.name})',
                      set(a+b for a in self.range for b in other.range))
        res.left = other
        res.right = self
        return res
    
    def __add__(self, other):
        return self.__radd__(other)
    
    def __rmul__(self, other):
        if isinstance(other, int):
            other = Symbol(f'{other}', {other})
        if self.is_zero or other.is_zero: return zero
        elif self.is_one: return other
        elif other.is_one: return self
        res = Symbol(f'({self.name} * {other.name})',
                      set(a*b for a in self.range for b in other.range))
        res.left = other
        res.right = self
        return res
    
    def __mul__(self, other):
        return self.__rmul__(other)
    
    def __floordiv__(self, other):
        if isinstance(other, int):
            other = Symbol(f'{other}', {other})
        if other.is_one: return self
        res = Symbol(f'({self.name} / {other.name})', 
                      set(int(a/b) for a in self.range for b in other.range))
        res.left = self
        res.right = other
        return res
       
    def __mod__(self, other):
        if isinstance(other, int):
            other = Symbol(f'{other}', {other})
        res = Symbol(f'({self.name} % {other.name})',
                  set(a%b for a in self.range for b in other.range))
        res.left = self
        res.right = other
        return res
    
    def __rmod__(self, other):
        return self.__mod__(other)
    
    def __eq__(self, other):
        if isinstance(other, int):
            other = Symbol(f'{other}', {other})
        if self.is_const and other.is_const:
            return one if self.const_value == other.const_value else zero
        
        if self.range & other.range:
            res = Symbol(f'[{self.name} == {other.name}]', {0, 1})
            res.left = self
            res.right = other
            return res
        else:
            return zero
        
zero = Symbol('0', {0})
one = Symbol('1', {1})

In [7]:
symbols = [Symbol(f'd{i}') for i in range(14)]

In [8]:
program = list(map(str.split, open('../input/day24')))

In [9]:
# def evaluate(n):
state = { 'w': 0, 'x': 0, 'y': 0, 'z': Symbol('0', {0})}
# input = map(int, n)
it = iter(symbols)

i = 0
for inst in program:
    match inst:
        case ['inp', x]:
            print(state)
            state[x] = next(it)
            state['z'] = Symbol(f'z{i}', set(random.choices(list(state['z'].range), k=1000)))
            i += 1
            input()
        case [op, a, b]:
            b = state[b] if b in state else int(b)
            match op:
                case 'add': state[a] = state[a] + b
                case 'mul': state[a] = state[a] * b
                case 'div': state[a] = state[a] // b
                case 'mod': state[a] = state[a] % b
                case 'eql': state[a] = state[a] == b
    # return state

{'w': 0, 'x': 0, 'y': 0, 'z': 0}


{'w': d0, 'x': 1, 'y': (d0 + 15), 'z': (d0 + 15)}


{'w': d1, 'x': 1, 'y': (d1 + 8), 'z': ((z1 * 26) + (d1 + 8))}


{'w': d2, 'x': 1, 'y': (d2 + 2), 'z': ((z2 * 26) + (d2 + 2))}


{'w': d3, 'x': [[((z3 % 26) + -9) == d3] == 0], 'y': ((d3 + 6) * [[((z3 % 26) + -9) == d3] == 0]), 'z': (((z3 / 26) * ((25 * [[((z3 % 26) + -9) == d3] == 0]) + 1)) + ((d3 + 6) * [[((z3 % 26) + -9) == d3] == 0]))}


...

{'w': d8, 'x': 1, 'y': (d8 + 5), 'z': ((z8 * 26) + (d8 + 5))}


{'w': d9, 'x': [[((z9 % 26) + -7) == d9] == 0], 'y': ((d9 + 13) * [[((z9 % 26) + -7) == d9] == 0]), 'z': (((z9 / 26) * ((25 * [[((z9 % 26) + -7) == d9] == 0]) + 1)) + ((d9 + 13) * [[((z9 % 26) + -7) == d9] == 0]))}


{'w': d10, 'x': [[((z10 % 26) + -12) == d10] == 0], 'y': ((d10 + 9) * [[((z10 % 26) + -12) == d10] == 0]), 'z': (((z10 / 26) * ((25 * [[((z10 % 26) + -12) == d10] == 0]) + 1)) + ((d10 + 9) * [[((z10 % 26) + -12) == d10] == 0]))}


{'w': d11, 'x': [[((z11 % 26) + -10) == d11] == 0], 'y': ((d11 + 6) * [[((z11 % 26) + -10) == d11] == 0]), 'z': (((z11 / 26) * ((25 * [[((z11 % 26) + -10) == d11] == 0]) + 1)) + ((d11 + 6) * [[((z11 % 26) + -10) == d11] == 0]))}


{'w': d12, 'x': [[((z12 % 26) + -1) == d12] == 0], 'y': ((d12 + 2) * [[((z12 % 26) + -1) == d12] == 0]), 'z': (((z12 / 26) * ((25 * [[((z12 % 26) + -1) == d12] == 0]) + 1)) + ((d12 + 2) * [[((z12 % 26) + -1) == d12] == 0]))}


In [12]:
state['z']

(((z13 / 26) * ((25 * [[((z13 % 26) + -11) == d13] == 0]) + 1)) + ((d13 + 2) * [[((z13 % 26) + -11) == d13] == 0]))